In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, SelectionSlider, IntSlider
from IPython.display import YouTubeVideo

# Unidad 2: Programación probabilística

## Contenidos de la unidad

- Introducción a los modelos generativos e inferencia bayesiana
- Método de Integración por Monte Carlo. Proceso de Markov y caminata aleatoria
- Algoritmos de muestreo por rechazo y Metropolis Hastings
- Markov Chain Monte Carlo: Inferencia y Diagnósticos



## ¿Qué hemos aprendido hasta ahora?

- Leyes fundamentales de probabilidades
- Variables aleatorias, distribuciones y estadísticos
- Ajuste de distribuciones y bondad de ajuste
- Números pseudo aleatorios
- Bootstrapping

## ¿Qué es simular?

- ["Hacer que algo parezca real no siéndolo"](https://www.rae.es/dpd/simular)
- Del latín [simulâre: "copiar, representar"](https://es.wiktionary.org/wiki/simulaci%C3%B3n) o [simulatio "acción y efecto de imitar algo"](http://etimologias.dechile.net/?simulacio.n)
- Reproducir artificialmente un fenómeno o las relaciones entrada-salida de un sistema


> Una simulación digital es la aplicación de un modelo computacional para la predicción de eventos físicos o el comportamiento de sistemas de ingeniería

<img src="images/paradigms.jpg" width="600">

## ¿Por qué simulamos en ingeniería y ciencia?

La simulación nos permite explorar nuevas teorías o diseñar nuevos experimentos para probar dichas teorias

Usando simulación podemos estudiar fenomenos que 
- no son observables
- las observaciones o mediciones son demasiado caras, peligrosas o impracticas de realizar (poca información)

Usando simulaciones podemos 
- analizar un sistema antes de haberlo construido
- analizar situaciones a las que el sistema aun no ha sido expuesto
- realizar predicciones sobre el comportamiento del sistema
- analizar la influencia e interelaciones entre las variables del sistema y en consencuencia entender mejor su operación. [Ceteris paribus](https://en.wikipedia.org/wiki/Ceteris_paribus)


Los simuladores son también ampliamente usados en educación

Con el rápido avance y la disminución en costos de la computación, la simulación digital se ha vuelto una herramienta clave en muchas disciplinas (eléctrica, mecánica, química, aeroespacial, nuclear, biomédica, materiales, etc)

> [Los principales desafíos de los simuladores están en la validación, verificación y medición de incerteza de los modelos](http://www.mcs-uab.com/docs/NSF.Simluation-Based_Engineering_Science.2006.pdf)



<img src="images/morpho.jpg" width="700">

## Modelo generativo

> [“What I cannot create, I do not understand.” Richard Feynman](https://jcs.biologists.org/content/joces/130/18/2941.full.pdf)

Sea un observación descrita por $x$ que viene de una distribución de probabilidad $p^*(x)$

Un modelo generativo es una aproximación $p_\theta(x) \approx p^*(x)$ con la cual podemos generar nuevos ejemplos  aleatorios de $x$


In [ ]:
import scipy.stats

with open("data/mistery_data.npy", "rb") as f:
    data = np.load(f) # Normal mu 10 y std 4

print(data)

dist = scipy.stats.norm # Seleccionar distribucion
args = dist.fit(data)   # Ajustar parámetros con MLE
print(args)

In [ ]:
mus = np.linspace(0, 20, num=20)
stds = np.linspace(2, 10, num=20)
logL = np.zeros(shape=(len(mus), len(stds)))
for i, mu in enumerate(mus):
    for j, std in enumerate(stds):
        logL[i, j]= sum([dist.logpdf(x, loc=mu, scale=std) for x in data])

In [ ]:
model = dist(*args) # Crear modelo
new_data = model.rvs(size=1000) # Generar datos
print(new_data[:10])

fig, ax = plt.subplots(1, 2, figsize=(6, 3), tight_layout=True)
ax[0].pcolormesh(stds, mus, np.exp(logL), cmap=plt.cm.Reds)
ax[1].hist(new_data, bins=20, density=True)
x = np.linspace(model.ppf(0.001), model.ppf(0.999))
ax[1].plot(x, model.pdf(x), 'k--');

Lamentablemente los ejemplos de la vida real son demasiado complejos para modelarse con distribuciones sencillas


In [ ]:
with open("data/mnist_data.npy", "rb") as f:
    data = np.load(f) 
    
dist = scipy.stats.multivariate_normal 
mu = np.mean(data.reshape(-1, 28*28), axis=0) # MLE de la media
cov = np.cov(data.reshape(-1, 28*28), rowvar=False) # MLE de la covarianza
model = dist(mean=mu, cov=cov+0.001*np.eye(28*28)) # Crear modelo
new_data = model.rvs(size=10) # Generar datos

fig, ax = plt.subplots(2, 10, tight_layout=True, figsize=(8, 2))
for i in range(10):
    ax[0, i].matshow(data[i], cmap=plt.cm.Greys_r)
    ax[0, i].axis('off')
    ax[1, i].matshow(new_data[i].reshape(28, 28), cmap=plt.cm.Greys_r)
    ax[1, i].axis('off')

Es mucho más conveniente asumir que existe una variable latente $z$ con una distribución sencilla $p(z)$ que luego se modifica a través de una transformación no lineal (e.g. red neuronal) $p(x|z)$ para obtener un ejemplo de $x$

En ese caso buscamos modelar la evidencia o verosimilitud marginal

$$
p(x) = \int_{\mathcal{z\in Z}} p(x|z) p(z) \,dz = \int_{\mathcal{z\in Z}} p(x, z) \,dz
$$

Esto además nos permite condicionar el sistema generador con variables que nos interesen

[Los modelos generativos se investigan muy activamente hoy en día](https://openai.com/blog/generative-models)

In [ ]:
YouTubeVideo('O1by05eX424')

## Recordatorio: Teorema de Bayes

De las propiedades de las probabilidades condicionales y la ley de probabilidades totales podemos escribir

$$
p(y|x) = \frac{p(x|y) p(y)}{p(x)} = \frac{p(x|y) p(y)}{\int p(x|y) p(y) \,dy}
$$

Tipicamente $x$ representa un conjunto de datos que hemos observado a través de un experimento e $y$ algún parámetro que queremos estimar. 

Discutamos: ¿Qué representan $p(y)$ y $p(y|x)$? ¿Cuándo es conveniente usar el teorema de bayes?



### Desafio

La marginalización de la variable latente, el cálculo de la esperanza de una variable continua o el cálculo de la evidencia en el teorema de Bayes requiere resolver integrales que pueden ser muy complicadas



## Métodos de Monte Carlo

Los métodos de Monte Carlo son una clase de métodos para resolver problemas matemáticos **usando muestras aleatorias** (o más bien pseudoaleatorias)

Los métodos de Monte Carlo se usan para predecir el comportamiento de un sistema en un escenario incierto (aleatorio)

Por ejemplo en la siguiente figura se predice el **valor esperado** del GDP per capita a un cierto horizonte

<img src="images/montecarlo1.png" width="500">

Pero también pueden usarse en casos completamente determinísticos

Por ejemplo en el caso de estimación de iluminación, la solución analítica es muchas veces infactible de calcular. En lugar de eso se puede aproximar usando una muestra aleatoria de rayos lanzados desde la fuente de iluminación

Si lanzamos "suficientes" rayos al azar entonces podemos modelar bien la iluminación real

<img src="images/montecarlo2.png" width="400">

¿Qué relación tiene esto con una encuesta de opinión?

## Breve historia de los métodos de Monte Carlo

En los 40s [Stanislaw Ulam](https://en.wikipedia.org/wiki/Stanislaw_Ulam), matemático polaco-américano, estaba en cama recuperandose de una enfermedad y pasaba el tiempo jugando solitario. Empezó a interesarse en calcular la probabilidad de ganar el juego de solitario. Trató de desarrollar las combinatorias sin éxito, era demasiado complicado

Luego pensó

> Supongamos que juego muchas manos, cuento las veces que gano y divido por la cantidad de manos jugadas

Sin embargo el había jugado muchas manos sin ganar, posiblemente le tomaría años hacer este conteo

Ulam pensó entonces en simular el juego usando un computador, por lo que recurrió a [John von Neumann](https://en.wikipedia.org/wiki/John_von_Neumann), quien implementó el algoritmo propuesto por Ulam en el [ENIAC](https://en.wikipedia.org/wiki/ENIAC)

Más adelante este algoritmo fue central en las simulaciones realizadas en el proyecto Manhattan. En aquel entonces [Nicholas Metropolis](https://en.wikipedia.org/wiki/Nicholas_Metropolis), colega de von Neumann y Ulam sugirió el nombre de Monte Carlo, haciendo alusión al famoso [casino de Monte Carlo](https://es.wikipedia.org/wiki/Casino_de_Montecarlo) que se encuentra en principado de Monaco en Europa.


## Esquema general  de un método de Monte Carlo

1. Se muestrean aleatoriamente las variables de entrada $X$
1. Se calcula una "cantidad de interés" $Y=g(x)$ (variable de salida)
1. Se reduce el resultando usando estadísticos, por ejemplo $\bar Y$ y $\sigma_Y$

La cantidad de interés es también una variable aleatoria. Es conveniente que las variables de entrada sigan una distribución sencilla (e.g. uniforme)

## Estimando el valor esperado de una función

El valor esperado de una función $g(x)$ es

$$
\mathbb{E}[g] = \int g(x) f(x) \,dx
$$

donde $f(x)$ es la densidad de $x$

Si la función y/o la integral son muy complicadas de calcular podemos en lugar de eso

- Muestrar aleatoriamente $N$ valores $x_i \sim f(x)$
- Evaluar $y_i = g(x_i)$
- aproximar el valor esperado como $V\frac{\sum_i y_i}{N}$

donde $V = \int f(x) \,dx$ es el volumen de integración

Cuando el método de Monte Carlo se usa para calcular un valor esperado se llama "Integración por Monte Carlo"

## Ejemplo: Calculando $\pi$ usando integración por Monte Carlo

El área es la integral definida de la función en su dominio

- La fórmula análitica del área de un cuadrado de lado $a$ es $a^2$
- La fórmula analítica del área de un circulo de radio $a$ es $\pi a^2$

Podemos estimar $\pi$ como el cociente $\frac{A_{circulo}}{A_{cuadrado}}$. Estimemos las áreas usando integración por Monte Carlo

- ¿Cómo cambia el resultado con $N$?
- ¿Qué ocurre si hay un sesgo al generar las muestras?

In [ ]:
def g(x1, x2):
    return (x1-0)**2 + (x2-0)**2 -1. <= 0.

N = 1000

x = np.random.rand(N, 2) # El volumen de integración es 1
N_inside = sum(g(x[:, 0], x[:, 1]))
print(4*N_inside/N)

In [ ]:
X1, X2 = np.meshgrid(np.linspace(0, 1, num=1000), 
                     np.linspace(0, 1, num=1000))
fig, ax = plt.subplots(figsize=(4, 3), tight_layout=True)
ax.contourf(X1, X2, g(X1, X2), cmap=plt.cm.RdBu_r); 
ax.set_aspect('equal')
ax.scatter(x[:, 0], x[:, 1], s=1, c='k', alpha=1);

In [ ]:
logN = np.arange(0, 7, step=0.1)
pi = np.zeros_like(logN)
errpi = np.zeros_like(pi)

for i, logn in enumerate(logN):
    xr = np.random.rand(int(10**logn), 2)
    pi[i] = 4.*np.mean(g(xr[:, 0], xr[:, 1]))    

fig, ax = plt.subplots(figsize=(4, 2), tight_layout=True)
ax.plot(logN, pi)
ax.axhline(np.pi, linestyle='--', color='r', alpha=.5);

## Experimento de lanzar una moneda

¿Cuál es la probabilidad de que el próximo lanzamiento sea cara?

- Si lancé la moneda una vez y salió cara
- Si lancé la moneda dos veces y en ambas salió cara
- Si lancé la moneda 100 veces y en todas salió cara
- Si lancé la moneda 100 veces y en 52 salió cara y 48 sello

¿Cómo es la varianza en cada caso? ¿Qué relación tiene que ver nuestra confianza sobre el resultado?

Notas para el futuro:
- Usar sólo un tipo de dígito, mostrar con más digitos y cambiar a una distribución acotada